In [55]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from scraping import Scraping
%load_ext autoreload
%autoreload 2
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# cleaning the function

In [2]:
def extract_clean(self):
        file = pdfplumber.open(self.pdf)
        if len(self.page) > 1 :
            joined_text = []
            for el in self.page:
                page = file.pages[el]
                text = page.extract_text()
                joined_text.append(text)
            text ='             NEXT          '.join(joined_text)
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', str(np.nan))
            text = re.sub('--', str(np.nan), text)
            if nltk.sent_tokenize(text) < 15:
                text = text.replace('\n','. ')
                # text = len_sentences(text)
                text = nltk.sent_tokenize(text)
                return text
            text = text.replace('\n',' ')
            # text = len_sentences(text)
            text = nltk.sent_tokenize(text)
            return text
        else:
            page = file.pages[self.page[0]]
            text = page.extract_text()
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', str(np.nan))
            text = re.sub('--', str(np.nan), text)
            if nltk.sent_tokenize(text) < 15:
                text = text.replace('\n','. ')
                # text = len_sentences(text)
                text = nltk.sent_tokenize(text)
                return text
            text = text.replace('\n',' ')
            # text = len_sentences(text)
            text = nltk.sent_tokenize(text)
            return text

In [3]:
ctt = pdfplumber.open('data/BancoCTT2021.pdf')
page = ctt.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

FileNotFoundError: [Errno 2] No such file or directory: 'data/BancoCTT2021.pdf'

In [ ]:
bankinter = pdfplumber.open('data/bankinter.pdf')
page = bankinter.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

In [ ]:
bai = pdfplumber.open('data/bancoBAIeurope2021com.pdf')
page = bai.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

In [ ]:
Scraping('data/bancoBAIeurope2021com.pdf', 4, '4').extract_clean()

# connecting name of product and name of commission

In [ ]:
abanca = Scraping('data/abanca.pdf', 4, 'abanca')
ctt = Scraping('data/ctt.pdf',4,'5')
bai = Scraping('data/bai.pdf',4,'4')
bic = Scraping('data/bic.pdf',5,'3')
bankinter = Scraping('data/bankinter.pdf',4,'2')

In [ ]:
ab_t = abanca.tokenize()
ab_n = abanca.names()

In [ ]:
ab_n

In [ ]:
com_dict.values()
    

In [ ]:
com_dict = {'Emissão de Extrato':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
           'Fotocópias e 2ªvias':['Fotocópias de segundas vias de talões de depósito',
                                  'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                 'Segundas vias (pedido na agência)'],
           'Manutenção de Conta':['Manutenção de conta', 'Comissão de manutenção de conta'],
           'Levantamento de Numerário':['Levantamento de numerário', 'Levantamento de numerário ao balcão'],
           'Adesão ao Serviço Online':['Adesão ao serviço de banca à distância', 'Adesão ao serviço online'],
            'Depósitos de Moedas':['Depósito de moedas metálicas', 'Depósito de moedas',
                                   'Depósito de moedas ao balcão', 'Depósito de dinheiro ao balcão',
                                  'Depósito de moeda metálica (≥ a 100 moedas)'],
            'Ateração de Titulares':['Alteração de titulares', 'Alteração de titularidade',
                                     'Alteração de titularidade / intervenientes'],
            'Descoberto Bancário':['Comissões por descoberto bancário', 'Descoberto bancário'],
            'Consulta de Movimentos':['Consulta de Movimentos de conta DO com', 'Consulta de movimentos ao balcão'],
            'Consulta de Saldo':['Pedido de saldo ao balcão', 'Consulta de Saldo de conta DO com comprovativo']
           }

In [ ]:
ab_t[:5]

In [ ]:
# keys = [x for x in com_dict.keys()]
values = [x for x in com_dict.values()]
lista ={}
for commission in values:
    for value in commission:
        for ind,sentence in enumerate(ab_t):
            if value in sentence:
                if value in lista:
                    if '[0-9]{1-2},[0-9]{2}' in sentence:
                        lista[value]= lista[value].append(sentence)
                    else:
                        lista[value]= lista[value].append(' '.join([sentence,file[ind+1]]))
                else:
                    if '[0-9]{1-2},[0-9]{2}' in sentence:
                        lista[value]= [sentence]
                    else:
                        lista[value]= [' '.join([sentence,ab_t[ind+1]])]
# return lista
                

In [ ]:
len(lista)

In [ ]:
abanca.values()

# how to join and search for the name of commision and account together

In [4]:
com_dict = {'Emissão de Extrato':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
           'Fotocópias e 2ªvias':['Fotocópias de segundas vias de talões de depósito',
                                  'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                 'Segundas vias (pedido na agência)'],
           'Manutenção de Conta':['Manutenção de conta', 'Comissão de manutenção de conta'],
           'Levantamento de Numerário':['Levantamento de numerário', 'Levantamento de numerário ao balcão'],
           'Adesão ao Serviço Online':['Adesão ao serviço de banca à distância', 'Adesão ao serviço online'],
            'Depósitos de Moedas':['Depósito de moedas metálicas', 'Depósito de moedas',
                                   'Depósito de moedas ao balcão', 'Depósito de dinheiro ao balcão',
                                  'Depósito de moeda metálica (≥ a 100 moedas)'],
            'Ateração de Titulares':['Alteração de titulares', 'Alteração de titularidade',
                                     'Alteração de titularidade / intervenientes'],
            'Descoberto Bancário':['Comissões por descoberto bancário', 'Descoberto bancário'],
            'Consulta de Movimentos':['Consulta de Movimentos de conta DO com', 'Consulta de movimentos ao balcão'],
            'Consulta de Saldo':['Pedido de saldo ao balcão', 'Consulta de Saldo de conta DO com comprovativo']
           }

In [5]:
ab_t = abanca.tokenize()
ab_n = abanca.names() 

NameError: name 'abanca' is not defined

In [6]:
results = {}
type_account = {}
for name in ab_n: 
    for key in com_dict:
        for values in key:
            print('object')
            for value in values:
                for ind,sentence in enumerate(ab_t):
                    if name in sentence and value in sentence[:ind+3]:
                        if name in type_account:
                            type_account[name].append(sentence)
                        else:
                            type_account[name]=[sentence]
                            print(type_account)

NameError: name 'ab_n' is not defined

In [7]:
type_account

{}

In [8]:
for ind,sentence in enumerate(ab_t):
    for name in ab_n:
        if name in sentence:
            results[name]=sentence

NameError: name 'ab_t' is not defined

In [9]:
ab_n

NameError: name 'ab_n' is not defined

In [10]:
ab_t

NameError: name 'ab_t' is not defined

In [11]:
Scraping('data/2_all_products.pdf', 4, '7').values()

TypeError: __init__() takes 3 positional arguments but 4 were given

In [12]:
Scraping('data/abanca.pdf', 4).values()

{'Emissão de extrato': ['Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2. Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3.'],
 'Fotocópias de segundas vias de talões de depósito': ['Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3. Manutenção de conta      3.1 Conta D.O.'],
 'Manutenção de conta': ['Manutenção de conta      3.1 Conta D.O. Particulares (Nota 2)      - Saldo médio até 1.000€  15,00 Trimestral  60,00  - Saldo médio entre 1.001€ e 2.500€  10,00 Trimestral  40,00  - Saldo médio superior a 2.500€  0,00  nan  3.2 Conta Ordenado (Nota 1 e Nota 16)  0,00  nan  3.3 Conta Standard (não admite novas contratações)  0,00  nan  3.4 Conta Future (Nota 3)  0,00  nan  3.5 Conta Kids (Nota 4)  0,00  nan  IS – 4%    3.6 Conta Global (Nota 5)  6,53 / 7,23 Mensal  85,14  3.7 Conta Base

In [13]:
ab_n

NameError: name 'ab_n' is not defined

In [14]:
lista =[]
for idx,name in enumerate(ab_n):
    single = ' '.join(name.split(" ")[:2])
#     print(single)
#     print(name[:1])
    if single not in lista:
        lista.append(name)
lista

NameError: name 'ab_n' is not defined

In [ ]:
Scraping('data/abanca.pdf',4).names()

In [43]:
# def dictionary:
output = {'bank_id':{'demand_depos':
                             {'example_subproduct':
                                          {'statement' : None,
                                           'documents_copy' : None,
                                           'acc_manteinance' : None,
                                           'withdraw': None,
                                           'online_service' : None,
                                           'cash_deposit': None,
                                           'change_holder' : None,
                                           'bank_overdraft': None,
                                           'movement_consultation': None,
                                           'balance_inquiry': None}},
                                     
                    'house_credit':{'subproduct':
                                            {'commissions': None}},
                     
                    'term_depos':{'subproduct' : 
                                              {'commission': None}}
                    }
         }


In [46]:
ab_output = {}
ab_output['demand_depos']=Scraping('data/abanca.pdf',4).demand_depos()
ab_output

{'demand_depos': {'Conta D.O.': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Ordenado': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Standard': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Future': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': N

In [17]:
subproduct_dict = {'statement' : None,
                   'documents_copy' : None,
                   'acc_manteinance' : None,
                   'withdraw': None,
                   'online_service' : None,
                   'cash_deposit': None,
                   'change_holder' : None,
                   'bank_overdraft': None,
                   'movement_consultation': None,
                   'balance_inquiry': None}

In [24]:
demand_depos = {}
abanca = Scraping('data/abanca.pdf',4)
for each in abanca.names():
    demand_depos[each] = subproduct_dict
demand_depos   

{'Conta D.O.': {'statement': None,
  'documents_copy': None,
  'acc_manteinance': None,
  'withdraw': None,
  'online_service': None,
  'cash_deposit': None,
  'change_holder': None,
  'bank_overdraft': None,
  'movement_consultation': None,
  'balance_inquiry': None},
 'Conta Ordenado': {'statement': None,
  'documents_copy': None,
  'acc_manteinance': None,
  'withdraw': None,
  'online_service': None,
  'cash_deposit': None,
  'change_holder': None,
  'bank_overdraft': None,
  'movement_consultation': None,
  'balance_inquiry': None},
 'Conta Standard': {'statement': None,
  'documents_copy': None,
  'acc_manteinance': None,
  'withdraw': None,
  'online_service': None,
  'cash_deposit': None,
  'change_holder': None,
  'bank_overdraft': None,
  'movement_consultation': None,
  'balance_inquiry': None},
 'Conta Future': {'statement': None,
  'documents_copy': None,
  'acc_manteinance': None,
  'withdraw': None,
  'online_service': None,
  'cash_deposit': None,
  'change_holder': Non

In [32]:
# loop =[]
# lista = []
# for n in demand_deposit:
#     for k,commission in com_dict.items():
#         sub_lista = []
#         sub_lista.append(n)
#         sub_lista.append(commission)
#     lista.append(sub_lista)
#     loop = filter(lista,abanca.tokenize())
# #             for idx,sentence in enumerate(abanca.tokenize()):
# #             if n in sentence:
# #                 text = abanca.tokenize()[idx:]
# #                 print('HEEEEEELP')
# #                 print(text)

In [70]:
# abanca.tokenize()

In [69]:
# Scraping('data/ctt.pdf',4).output()

In [68]:
# Scraping('data/bai.pdf',4).output()

In [67]:
# Scraping('data/bankinter.pdf',4).output()

In [66]:
# Scraping('data/bic.pdf',5).output()

In [65]:
remote = urlopen(Request("https://storage.googleapis.com/bank_price_pdfs/4_all_products.pdf")).read()
memory = BytesIO(remote)
De("https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf", [4]).output()

{'demand_depos': {'Conta D.O.': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Ordenado': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Standard': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': None,
   'cash_deposit': None,
   'change_holder': None,
   'bank_overdraft': None,
   'movement_consultation': None,
   'balance_inquiry': None},
  'Conta Future': {'statement': None,
   'documents_copy': None,
   'acc_manteinance': None,
   'withdraw': None,
   'online_service': N

In [58]:
memory